<a href="https://colab.research.google.com/github/muajnstu/DSK-Chain-to-predict-diabeties-/blob/main/Evaluate_Deep_Learning_Baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, Add
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/DSK-Chain-to-predict-diabeties-/refs/heads/main/Cleaned%20data%20after%20removal%20of%20duplicate%20values.csv')

X = df.drop(columns=['Outcome'])
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=46, stratify=y
)
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (5308, 28)
Test shape: (1328, 28)


In [ ]:
def print_metrics(y_true, y_pred, y_prob=None):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    num_classes = cm.shape[0]

    if num_classes == 2:
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        gmean = np.sqrt(specificity * sensitivity)
        type1 = FP / (FP + TN) if (FP + TN) > 0 else 0
        type2 = FN / (TP + FN) if (TP + FN) > 0 else 0
        fmeasure = f1_score(y_true, y_pred, pos_label=1)
        auc = 0
        if y_prob is not None:
            try:
                auc = roc_auc_score(y_true, y_prob)
            except Exception as e:
                print(f"AUROC calculation error: {e}")
                auc = 0
    else:
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)
        specificity = np.mean([TN[i] / (TN[i] + FP[i]) if (TN[i] + FP[i]) > 0 else 0 for i in range(num_classes)])
        sensitivity = np.mean([TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        gmean = np.sqrt(specificity * sensitivity)
        type1 = np.mean([FP[i] / (FP[i] + TN[i]) if (FP[i] + TN[i]) > 0 else 0 for i in range(num_classes)])
        type2 = np.mean([FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        fmeasure = f1_score(y_true, y_pred, average='macro')
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
            except Exception as e:
                print(f"AUROC calculation error: {e}")
                auc = 0

    print(f"Accuracy      : {accuracy:.4f}")
    print(f"Sensitivity   : {sensitivity:.4f}")
    print(f"Specificity   : {specificity:.4f}")
    print(f"G-Mean        : {gmean:.4f}")
    print(f"Type I Error  : {type1:.4f}")
    print(f"Type II Error : {type2:.4f}")
    print(f"F1 Score      : {fmeasure:.4f}")
    print(f"AUROC         : {auc:.4f}")


MLP & Res-MLP

In [ ]:
def make_mlp(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes if num_classes > 2 else 1, activation='softmax' if num_classes > 2 else 'sigmoid'))
    model.compile(
        optimizer=Adam(0.001),
        loss='categorical_crossentropy' if num_classes > 2 else 'binary_crossentropy',
        metrics=['accuracy']
    )
    return model

print("\n--- MLP ---")
mlp = make_mlp(input_dim=X_train.shape[1], num_classes=len(np.unique(y_train)))
mlp.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
y_prob = mlp.predict(X_test).flatten()
y_pred = (y_prob > 0.5).astype(int)
print_metrics(y_test, y_pred, y_prob)


# Residual MLP
def make_residual_mlp(input_dim, num_classes):
    inp = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inp)
    x = Dropout(0.3)(x)
    x1 = Dense(64, activation='relu')(x)
    x2 = Dense(64, activation='relu')(x1)
    x2 = Add()([x1, x2])
    out = Dense(num_classes if num_classes > 2 else 1, activation='softmax' if num_classes > 2 else 'sigmoid')(x2)
    model = Model(inp, out)
    model.compile(
        optimizer=Adam(0.001),
        loss='categorical_crossentropy' if num_classes > 2 else 'binary_crossentropy',
        metrics=['accuracy']
    )
    return model

print("\n--- Residual MLP ---")
resmlp = make_residual_mlp(input_dim=X_train.shape[1], num_classes=len(np.unique(y_train)))
resmlp.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
y_prob = resmlp.predict(X_test).flatten()
y_pred = (y_prob > 0.5).astype(int)
print_metrics(y_test, y_pred, y_prob)


--- MLP ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy      : 0.8148
Sensitivity   : 0.0000
Specificity   : 1.0000
G-Mean        : 0.0000
Type I Error  : 0.0000
Type II Error : 1.0000
F1 Score      : 0.0000
AUROC         : 0.6421

--- Residual MLP ---
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy      : 0.8155
Sensitivity   : 0.0081
Specificity   : 0.9991
G-Mean        : 0.0901
Type I Error  : 0.0009
Type II Error : 0.9919
F1 Score      : 0.0161
AUROC         : 0.6556


TabNet

In [ ]:
!pip install pytorch-tabnet

  Using cached pytorch_tabnet-4.1.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
print("\n--- TabNet ---")
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
tabnet = TabNetClassifier()
tabnet.fit(
    X_train.values, y_train_enc,
    max_epochs=20, patience=5, batch_size=32,
    eval_set=[(X_test.values, y_test_enc)],
    eval_metric=['auc']
)
y_prob = tabnet.predict_proba(X_test.values)[:, 1]
y_pred = tabnet.predict(X_test.values)
print_metrics(y_test_enc, y_pred, y_prob)


--- TabNet ---


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.50882 | val_0_auc: 0.55542 |  0:00:02s
epoch 1  | loss: 0.48337 | val_0_auc: 0.57933 |  0:00:05s
epoch 2  | loss: 0.4808  | val_0_auc: 0.55851 |  0:00:09s
epoch 3  | loss: 0.47998 | val_0_auc: 0.58929 |  0:00:12s
epoch 4  | loss: 0.47521 | val_0_auc: 0.62193 |  0:00:14s
epoch 5  | loss: 0.47351 | val_0_auc: 0.57992 |  0:00:17s
epoch 6  | loss: 0.475   | val_0_auc: 0.60029 |  0:00:21s
epoch 7  | loss: 0.47074 | val_0_auc: 0.58558 |  0:00:24s
epoch 8  | loss: 0.47336 | val_0_auc: 0.5946  |  0:00:27s
epoch 9  | loss: 0.47756 | val_0_auc: 0.56985 |  0:00:29s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_val_0_auc = 0.62193


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Accuracy      : 0.8148
Sensitivity   : 0.0000
Specificity   : 1.0000
G-Mean        : 0.0000
Type I Error  : 0.0000
Type II Error : 1.0000
F1 Score      : 0.0000
AUROC         : 0.6219


In [ ]:
!pip install tensorflow_decision_forests --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.20.0
    Uninstalling tensorboard-2.20.0:
      Successfully uninstalled tensorboard-2.20.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.20.0
    Uninstalling tensorflow-2.20.0:
      Successfully uninstalled tensorflow-2.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-tabular 1.1.1 requires protobuf<5.29.0,>=3.20.0, but you have protobuf

GrowNet & NODE

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/DSK-Chain-to-predict-diabeties-/refs/heads/main/Cleaned%20data%20after%20removal%20of%20duplicate%20values.csv')
X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=46, stratify=y
)

# TF datasets
train_df = X_train.copy()
train_df["Outcome"] = y_train
test_df = X_test.copy()
test_df["Outcome"] = y_test

# Use validation split for early stopping
val_df = train_df.sample(frac=0.15, random_state=42)
train_df = train_df.drop(val_df.index)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="Outcome")
val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_df, label="Outcome")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="Outcome")

def print_metrics(y_true, y_pred, y_prob=None):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    num_classes = cm.shape[0]

    if num_classes == 2:
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        gmean = np.sqrt(specificity * sensitivity)
        type1 = FP / (FP + TN) if (FP + TN) > 0 else 0
        type2 = FN / (TP + FN) if (TP + FN) > 0 else 0
        fmeasure = f1_score(y_true, y_pred, pos_label=1)
        auc = 0
        if y_prob is not None:
            try:
                auc = roc_auc_score(y_true, y_prob)
            except:
                auc = 0
    else:
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)
        specificity = np.mean([TN[i] / (TN[i] + FP[i]) if (TN[i] + FP[i]) > 0 else 0 for i in range(num_classes)])
        sensitivity = np.mean([TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        gmean = np.sqrt(specificity * sensitivity)
        type1 = np.mean([FP[i] / (FP[i] + TN[i]) if (FP[i] + TN[i]) > 0 else 0 for i in range(num_classes)])
        type2 = np.mean([FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        fmeasure = f1_score(y_true, y_pred, average='macro')
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
            except:
                auc = 0

    print(f"Accuracy      : {accuracy:.4f}")
    print(f"Sensitivity   : {sensitivity:.4f}")
    print(f"Specificity   : {specificity:.4f}")
    print(f"G-Mean        : {gmean:.4f}")
    print(f"Type I Error  : {type1:.4f}")
    print(f"Type II Error : {type2:.4f}")
    print(f"F1 Score      : {fmeasure:.4f}")
    print(f"AUROC         : {auc:.4f}")

# NODE
print("\n--- NODE-like (GradientBoostedTrees) ---")
node_model = tfdf.keras.GradientBoostedTreesModel(num_trees=200,
    max_depth=6

)
node_model.fit(train_ds, validation_data=val_ds)

y_pred_probs = node_model.predict(test_ds).squeeze()
y_pred = (y_pred_probs > 0.5).astype(int)
print_metrics(y_test, y_pred, y_pred_probs)

# GrowNet
print("\n--- GrowNet-like (RandomForest) ---")
grownet_model = tfdf.keras.RandomForestModel()
grownet_model.fit(train_ds, validation_data=val_ds)

y_pred_probs = grownet_model.predict(test_ds).squeeze()
y_pred = (y_pred_probs > 0.5).astype(int)
print_metrics(y_test, y_pred, y_pred_probs)


--- NODE-like (GradientBoostedTrees) ---
Use /tmp/tmpxdtynx7y as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.468839. Found 4512 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(796, shape=(), dtype=int32)
Validation dataset read in 0:00:00.417267. Found 796 examples.
Training model...
Model trained in 0:00:00.953031
Compiling model...


Model compiled.


2/2 [==============================] - 0s 10ms/step
Accuracy      : 0.8253
Sensitivity   : 0.0813
Specificity   : 0.9945
G-Mean        : 0.2843
Type I Error  : 0.0055
Type II Error : 0.9187
F1 Score      : 0.1471
AUROC         : 0.6969

--- GrowNet-like (RandomForest) ---
Use /tmp/tmpl13yjwbh as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.773030. Found 4512 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(796, shape=(), dtype=int32)
Validation dataset read in 0:00:00.610622. Found 796 examples.
Training model...
Model trained in 0:00:04.514398
Compiling model...
Model compiled.
2/2 [==============================] - 0s 14ms/step
Accuracy      : 0.8238
Sensitivity   : 0.0610
Specificity   : 0.9972
G-Mean        : 0.2466
Type I Error  : 0.0028
Type II Error : 0.9390
F1 Score      : 0.1136
AUROC         : 0.5837


DNCV2 & AutoInt